In [1]:
!pip install langchain
!pip install openai
!pip install seaborn
!pip install matplotlib
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install tiktoken
!pip install flair
!pip install quantulum3
!pip install pyap
!pip install stemming

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user



KeyboardInterrupt



In [ ]:
from tqdm.notebook import tqdm


In [ ]:
import json
with open("../scripts/datascraping/data/flyers_text.json", 'r') as file:
    data = json.load(file)

docs = []
for title, doc in data.items():
    for header, section in data.items():
        docs.append(str({title: {header: section}}))


In [ ]:
import random
import pandas as pd
docs = [doc for doc in docs if len(doc) < 3000]
sample = random.sample(docs, 50)

data = pd.DataFrame({"documents": sample})

data.head()

In [ ]:
from extract import extract_all_entities

data['entities'] = data['documents'].apply(extract_all_entities)

data.head()


In [ ]:
data.to_csv("entities.csv")



In [ ]:
from ner import extract_entities

# Applying the function to the 'documents' column
data['Entities'] = data['documents'].apply(extract_entities)

data
data.to_csv("entities.csv")


In [ ]:
from fact_qa import FactQAGenerator
from prompt_factory import PromptFactory
import langchain

template = '''
Gedraag je als iemand die online informatie zoekt en stelt vragen gebaseerd op specifieke feiten zoals locatie, telefoonnummer, naam, enzovoort.

Patiënten of gebruikers kunnen bijvoorbeeld vragen stellen als:
{{
    "qa_list": [
        {{
            "question": "Wat is het telefoonnummer van de kliniek voor noodgevallen?",
            "answer": "Het telefoonnummer van de kliniek voor noodgevallen is 012-345-6789."
        }},
        {{
            "question": "Waar is de hoofdingang van het ziekenhuis gelegen?",
            "answer": "De hoofdingang van het ziekenhuis is gelegen aan de Eerste Gezondheidsstraat 123 in Amsterdam."
        }}
        // meer vragen en antwoorden gebaseerd op specifieke feiten
    ]
}}

in het json formaat:
{format}

Document Informatie:
{document}

Belangrijke Opmerking: Bij het genereren van vragen, gebruik specifieke termen en benamingen uit het document in plaats van algemene termen. Verwijs direct naar de specifieke feit (zoals locatie, telefoonnummer, naam) om nauwkeurigheid en duidelijkheid in de vragen te waarborgen. Vermijd algemeenheden.

Maak vragen die beantwoord kunnen worden met de volgende informatie: {fact}

Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het formaat:
{format}

Zorg ervoor dat elke vraag en antwoord paar in een geldig JSON-formaat is. Dit betekent dat vragen en antwoorden tussen dubbele aanhalingstekens moeten staan, en de algemene structuur moet overeenkomen met het vereiste JSON-schema.
        '''

base_prompt = langchain.PromptTemplate(
            template=template,
            input_variables=['fact', 'n', 'document', 'format']
        )

num_questions_per_doc = 5

dfs = []

FactQAGenerator = FactQAGenerator()
# Generate the data
for index, row in data.iterrows():
    doc = row['documents']
    # Assuming phone numbers are separated by commas
    phone_numbers = row['phone_numbers'].split(',')
    try:
        for phone_number in phone_numbers:
            # Trimming any extra whitespace
            phone_number = phone_number.strip()
            fact = {'type': 'phone_number', 'value': phone_number}
            # Generate prompt for each document and phone number
            prompt = base_prompt.partial(
                n=num_questions_per_doc,
                document=doc)
            qa_list = FactQAGenerator.generate_question_for_fact(prompt, fact)
            for qa in qa_list:
                new_row = {'Document': doc, 'Question': qa.question, 'Answer': qa.answer, 'Fact': phone_number}
                dfs.append(new_row)
    except Exception as e:
            print(f"Failed to generate QA for document: {doc}. Error: {e}")
            continue

# Convert to DataFrame
df = pd.DataFrame(dfs)

# Display the DataFrame
print(df)


In [ ]:
df.to_csv('./fact-output.csv')

In [ ]:
from phone import normalize_phone_number

df['Normalized Fact'] = df['Fact'].apply(lambda x: normalize_phone_number(x) if x is not None else None)
df['Normalized Answer'] = df['Answer'].apply(lambda x: extract_phone_numbers(x))

# Filtering the DataFrame where normalized 'Answer' contains the normalized 'Fact'
filtered_df = df[df.apply(lambda x: x['Normalized Fact'] is not None and x['Normalized Fact'] in x['Normalized Answer'], axis=1)]
filtered_df


In [ ]:
df